In [10]:
import sys
fileDir = "/home/jovyan/notebooks"
sys.path.append(fileDir)

from utilities import *
from github import Github
import json
from github import Auth
import pandas as pd
import csv

In [12]:
session = SparkSession.builder \
    .appName("Get Sample") \
    .master(SPARK_MASTER_URL) \
    .config("spark.executor.memory", "5G") \
    .config("spark.authenticate", "false") \
    .getOrCreate()


In [13]:
HDFS_URL = "hdfs://namenode:9000//data-team"
PREFIX = "sample_"  # "sample_" or ""

### Reading from HDFS

In [14]:
# cambia se ti serve il dataset completo
repositories = session.read.json(f"{HDFS_URL}/{PREFIX}repositories.json")
repositories.printSchema()

root
 |-- repo_name: string (nullable = true)
 |-- watch_count: string (nullable = true)



In [25]:
# Import github token from json file
credentials = json.load(open(f'{fileDir}/credentials.json'))

# using an access token
auth = Auth.Token(credentials['token'])

# Public Web Github
g = Github(auth=auth)